<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/restructure_code/StarpilotProject_Implementing_evalutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 80kB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 665kB 48.9MB/s 
     |████████████████████████████████| 204kB 51.3MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 3.3MB 45.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 123 (delta 69), reused 20 (delta 0), pack-reused 0
Receiving objects: 100% (123/123), 758.44 KiB | 11.15 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# TESTING IMPLEMENTATION

In [5]:
from baseline import set_hyperparameters
import hyperparameters as h
from ppo import PPO
import torch

In [6]:
set_hyperparameters(baseline='Procgen')
model = PPO(print_output=True)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15


In [7]:
policy = model.train()

Step: 8192	Mean reward: 3.875
Step: 16384	Mean reward: 4.15625
Step: 24576	Mean reward: 5.0625
Step: 32768	Mean reward: 5.71875
Step: 40960	Mean reward: 5.03125
Step: 49152	Mean reward: 5.3125
Step: 57344	Mean reward: 5.71875
Step: 65536	Mean reward: 5.125
Step: 73728	Mean reward: 6.375


KeyboardInterrupt: ignored

In [8]:
tools.create_index_table_from_txt_files()

,file_name,total_steps,num_envs,num_levels,num_steps,num_epochs,batch_size,eps,grad_eps,value_coef,entropy_coef,lr,opt_extra,gamma,lmbda,version,optimizer,Time limit,Time spent (in seconds),Steps taken,Done,time_limit,Time spent (in seconds),Steps taken,Done,Last Mean Reward
Value,Baseline(PPO)_Run_15Nov_13h11m00s,8000000.0,32,10,128,3,256,0.1,0.5,1,0.01,0.00025,1e-05,0.99,0.95,Baseline(PPO),Adam,12:30:0,11253.99,8003584,True,NaN,NaN,NaN,NaN,10.0625
Value,Baseline(Procgen)_Run_15Nov_16h18m34s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,1e-05,0.999,0.95,Baseline(Procgen),Adam,12:30:0,10962.17,8003584,True,NaN,NaN,NaN,NaN,19.71875
Value,Baseline(PPO)_Run_16Nov_20h08m12s,8000000.0,32,10,128,3,256,0.1,0.5,1,0.01,0.00025,1e-05,0.99,0.95,Baseline(PPO),Adam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0625
Value,Baseline(Procgen)_Run_17Nov_14h07m08s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,NaN,0.999,0.95,Baseline(Procgen),NaN,NaN,NaN,NaN,NaN,43200,16572.48,8003584,False,20.1875
Value,Baseline(Procgen)_Run_21Nov_12h43m19s,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,NaN,0.999,0.95,Baseline(Procgen),NaN,NaN,NaN,NaN,NaN,43200,NaN,NaN,NaN,6.375


# Testing evaluation

In [67]:
from model import Encoder
from policy import Policy
import utils
import hyperparameters as h

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

feature_dim= 512    # <- The only thing we chose ourself

#Fixed values
in_channels = 3 #RGB
num_actions = 15  #Number of actions in the Procgen environment


MODEL_PATH = '/content/drive/My Drive/02456-Deep-Learning-Project/Models/'

def load_policy(file_name):
  encoder = Encoder(in_channels = in_channels, feature_dim = feature_dim)  
  
  policy = Policy(encoder = encoder, feature_dim = feature_dim, num_actions = num_actions)
  policy.cuda()
  policy.load_state_dict(torch.load(MODEL_PATH + file_name + '.pt'))
  policy.cuda() 

  return policy

In [68]:
def policy_evaluation(video_name='vid', print_output=False, policy=None, file_name=None):
  if policy is None and file_name is None:
    raise ValueError("Policy should be specified by file_name or given as object")
  if policy is None:
    policy = load_policy(file_name)
  
  # Make evaluation environment
  eval_env = utils.make_env(h.num_envs, start_level=h.num_levels, num_levels=h.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  for _ in range(512):  #<--- Hardcoded batch size?

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

In [69]:
#set_hyperparameters(baseline='Procgen')

INDEX_TABLE = tools.create_index_table_from_txt_files()

file_name = INDEX_TABLE['file_name'][2].strip()
print(file_name)
loaded_policy = load_policy(file_name=file_name)
policy_evaluation(print_output=True, policy = loaded_policy)
policy_evaluation(print_output=True, file_name=file_name)

Baseline(PPO)_Run_16Nov_20h08m12s
Average return: tensor(15.9751)
Average return: tensor(15.9751)


tensor(15.9751)